## Child_Detector 

Código baseado nos projetos: https://github.com/misbah4064/age_and_gender_detection.git , https://www.learnopencv.com/age-gender-classification-using-opencv-deep-learning-c-python/ .
Para rodar basta descomentar, tirar o # do início da linha, os comandos abaixo e executar uma única vez. Os comandos são para ambiente linux, caso esteja em outro sistema, por favor baixe e descompacte manualmente.

In [1]:
# Downloading pretrained data and unzipping it
#!wget https://drive.google.com/uc?id=1_aDScOvBeBLCn_iv0oxSO8X1ySQpSbIS

#!unzip modelNweight.zip

# !pipenv install
# !pipenv shell


In [2]:
# Import required modules
import cv2 as cv
import math
import time
import numpy as np
# from google.colab.patches import cv2_imshow

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes

faceProto = "modelNweight/opencv_face_detector.pbtxt"
faceModel = "modelNweight/opencv_face_detector_uint8.pb"

ageProto = "modelNweight/age_deploy.prototxt"
ageModel = "modelNweight/age_net.caffemodel"

genderProto = "modelNweight/gender_deploy.prototxt"
genderModel = "modelNweight/gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
genderList = ['Masculino', 'Feminino']

# Load network
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

padding = 20

def age_gender_detector(frame):
    # Read frame
    t = time.time()
    frameFace, bboxes = getFaceBox(faceNet, frame)
    for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]
        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        if (agePreds[0].argmax()< 3):
            label = "Crianca {}".format(gender)
            cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
            return frameFace, True
        else:
            return frame, False
    

In [3]:
input = cv.imread("images/baby1.jpg")
output, is_Child = age_gender_detector(input)
print("É criança? {}".format(is_Child))
cv.imshow("",output)
cv.waitKey(0)
cv.destroyAllWindows()

É criança? True


In [4]:
input = cv.imread("images/image.jpg")
output, is_Child = age_gender_detector(input)
print("É criança? {}".format(is_Child))
cv.imshow("",output)
cv.waitKey(0)
cv.destroyAllWindows()

É criança? False


In [5]:
input = cv.imread("images/teen1.jpg")
output, is_Child = age_gender_detector(input)
print("É criança? {}".format(is_Child))
cv.imshow("",output)
cv.waitKey(0)
cv.destroyAllWindows()

É criança? False


In [6]:
input = cv.imread("images/crianca2.jpg")
output, _ = age_gender_detector(input)
cv.imshow("",output)
cv.waitKey(0)
cv.destroyAllWindows()

In [7]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
# ret, frame = cap.read()
print("Processing Video...")
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        try:
            output, _ = age_gender_detector(frame)
            cv.imshow("Camera",output)
            key = cv.waitKey(30)
            if (key == 13): # Enter para sair
                break
        except:
            pass
cap.release()
cv.destroyAllWindows()
print("Done processing video")

Processing Video...
Done processing video
